### Load as pandas df

In [1]:
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest

In [2]:
df_adclick=pd.read_csv("ad_click_dataset.csv")
df_adclick

,id,full_name,age,gender,device_type,ad_position,browsing_history,time_of_day,click
0,670,User670,22.0,NaN,Desktop,Top,Shopping,Afternoon,1
1,3044,User3044,NaN,Male,Desktop,Top,NaN,NaN,1
2,5912,User5912,41.0,Non-Binary,NaN,Side,Education,Night,1
3,5418,User5418,34.0,Male,NaN,NaN,Entertainment,Evening,1
4,9452,User9452,39.0,Non-Binary,NaN,NaN,Social Media,Morning,0
...,...,...,...,...,...,...,...,...,...
9995,8510,User8510,NaN,NaN,Mobile,Top,Education,NaN,0
9996,7843,User7843,NaN,Female,Desktop,Bottom,Entertainment,NaN,0
9997,3914,User3914,NaN,Male,Mobile,Side,NaN,Morning,0
9998,7924,User7924,NaN,NaN,Desktop,NaN,Shopping,Morning,1


In [3]:
print(df_adclick.isnull().sum()) 

id                     0
full_name              0
age                 4766
gender              4693
device_type         2000
ad_position         2000
browsing_history    4782
time_of_day         2000
click                  0
dtype: int64


Here we need to handle these NaN values 
### Cleaning and pre-processing

In [4]:
used_columns = ['age', 'gender', 'device_type', 'ad_position', 'browsing_history', 'time_of_day', 'click']
#these are the columns we need, as instructed by TA, we will drop rows with NaN 

df_adclick = df_adclick.dropna(subset=used_columns)

df_adclick = df_adclick.drop(['id', 'full_name'], axis=1)

categorical_cols = ['gender', 'device_type', 'ad_position', 'browsing_history', 'time_of_day']
df_adclick = pd.get_dummies(df_adclick, columns=categorical_cols, drop_first=True)

In [5]:
df_adclick.info()

<class 'pandas.core.frame.DataFrame'>
Index: 816 entries, 17 to 9999
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   age                             816 non-null    float64
 1   click                           816 non-null    int64  
 2   gender_Male                     816 non-null    bool   
 3   gender_Non-Binary               816 non-null    bool   
 4   device_type_Mobile              816 non-null    bool   
 5   device_type_Tablet              816 non-null    bool   
 6   ad_position_Side                816 non-null    bool   
 7   ad_position_Top                 816 non-null    bool   
 8   browsing_history_Entertainment  816 non-null    bool   
 9   browsing_history_News           816 non-null    bool   
 10  browsing_history_Shopping       816 non-null    bool   
 11  browsing_history_Social Media   816 non-null    bool   
 12  time_of_day_Evening             816 non

In [6]:
df_adclick.head()

,age,click,gender_Male,gender_Non-Binary,device_type_Mobile,device_type_Tablet,ad_position_Side,ad_position_Top,browsing_history_Entertainment,browsing_history_News,browsing_history_Shopping,browsing_history_Social Media,time_of_day_Evening,time_of_day_Morning,time_of_day_Night
17,56.0,1,False,False,False,True,False,False,False,True,False,False,False,True,False
25,43.0,1,True,False,False,True,False,False,False,False,False,False,False,False,False
33,37.0,0,True,False,True,False,False,True,False,True,False,False,True,False,False
52,49.0,1,True,False,True,False,False,True,False,True,False,False,False,True,False
102,59.0,0,False,False,False,False,False,False,False,False,False,True,False,True,False


In [7]:
df_adclick.columns

Index(['age', 'click', 'gender_Male', 'gender_Non-Binary',
       'device_type_Mobile', 'device_type_Tablet', 'ad_position_Side',
       'ad_position_Top', 'browsing_history_Entertainment',
       'browsing_history_News', 'browsing_history_Shopping',
       'browsing_history_Social Media', 'time_of_day_Evening',
       'time_of_day_Morning', 'time_of_day_Night'],
      dtype='object')

### Splitting Dataset 

In [8]:
group_A = df_adclick[df_adclick['ad_position_Top'] == 1]  
group_B = df_adclick[df_adclick['ad_position_Side'] == 1]  

success_A = group_A['click'].sum()  
trials_A = group_A.shape[0]  

success_B = group_B['click'].sum()  
trials_B = group_B.shape[0]  

successes = [success_A, success_B]
trials = [trials_A, trials_B]

### Using statsmodels for z-test and printing 

In [9]:
successes = [success_A, success_B]
trials = [trials_A, trials_B]

z_score, p_value = proportions_ztest(successes, trials)

print(f"Z-score: {z_score}")
print(f"P-value: {p_value}")

Z-score: 1.2130583676113753
P-value: 0.22510750571860028


### Explaination 

#### 1. Z-score: 1.21
The Z-score of 1.21 indicates that the difference between the two proportions (click-through rates) is about 1.21 standard deviations away from the mean under the null hypothesis.

This means there's a mild difference between the click-through rates of the two groups (Top ad position vs. Side ad position), but it's not strong enough to be considered statistically significant.

#### 2. P-value: 0.225
The P-value is 0.225, which is greater than the conventional significance level of 0.05.

In hypothesis testing, if the P-value > 0.05, we fail to reject the null hypothesis.
This means the observed difference in click-through rates could very likely be due to random variation.

#### Conclusion:

Z-score: 1.21 suggests a modest difference, but not one that's statistically relevant to us.

P-value: 0.225 indicates that the difference in click-through rates is not statistically significant.

#### What Does This Mean for our A/B Test?

Since the P-value is greater than 0.05, we do not have sufficient evidence to conclude that the ad position (Top vs. Side) significantly affects the click-through rate.

In other words, the difference observed in this test could easily be explained by chance, and both ad positions perform similarly in terms of click-through rate.

These results are **statistically insignificant** and do not support a strong preference for one ad position over the other.

### Extra math and logic for self understanding

The Z-score (also called the standard score) is a statistical measure that describes how many standard deviations a data point or sample statistic is from the mean of a distribution. It is used to standardize values so that they can be compared across different distributions.

Formula for the Z-score:
The Z-score is calculated using the following formula:
$$
𝑍 = \frac{𝑋 − 𝜇} {𝜎}
$$​

### Z-test for Two Proportions

The formula for the Z-test for two proportions is:

$$
Z = \frac{(p_1 - p_2)}{\sqrt{P(1 - P)\left(\frac{1}{n_1} + \frac{1}{n_2}\right)}}
$$

Where:
- $p_1$ and $p_2$ are the sample proportions of the two groups.
- $P$ is the pooled proportion, calculated as:

$$
P = \frac{x_1 + x_2}{n_1 + n_2}
$$

- $n_1$ and $n_2$ are the sample sizes of the two groups.
- $x_1$ and $x_2$ are the number of successes in each group.

